# Fiddler Guardrails Free Trial Quick Start Guide




In [ ]:
%pip install -q matplotlib numpy pandas requests scikit-learn

import time
from typing import Tuple


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
from sklearn.metrics import (
    PrecisionRecallDisplay,
    RocCurveDisplay,
    classification_report,
)


In [ ]:
plt.style.use("ggplot")


In [ ]:
FIDDLER_GUARDRAILS_TOKEN = ""
GUARDRAILS_BASE_URL = "https://guardrails.cloud.fiddler.ai/v3/guardrails"

FAITHFULNESS_GUARDRAILS_URL = f"{GUARDRAILS_BASE_URL}/ftl-response-faithfulness"

GUARDRAILS_HEADERS = {
    "Authorization": f"Bearer {FIDDLER_GUARDRAILS_TOKEN}",
    "Content-Type": "application/json",
}

## 1. Load RAGTruth QA Dataset

Download from GitHub and set columns we'll use for looking at performance

In [ ]:
def read_ragtruth_qa_from_github(sample_size: int = None) -> pd.DataFrame:
    """Read RAGTruth from GitHub and filter to the Q&A examples in the test set.

    Args:
        sample_size (int, optional): How many records to return. If specified then a balanced sample will returned.
            Defaults to None, which means all records will be returned

    Returns:
        pd.DataFrame: RAGTruth dataset filtered to Q&A examples in the test set, optionally downsampled.
    """
    ragtruth_source_info = pd.read_json(
        "https://github.com/ParticleMedia/RAGTruth/raw/refs/heads/main/dataset/source_info.jsonl",
        lines=True,
    )
    ragtruth_response = pd.read_json(
        "https://github.com/ParticleMedia/RAGTruth/raw/refs/heads/main/dataset/response.jsonl",
        lines=True,
    )
    df = pd.merge(
        ragtruth_source_info,
        ragtruth_response,
        left_on="source_id",
        right_on="source_id",
    )
    df_qa = df[(df["task_type"] == "QA") & (df["split"] == "test")]

    if sample_size is None:
        return df_qa
    else:
        df_qa_pos = df_qa[df_qa["labels"].apply(len) == 0].sample(
            sample_size // 2, random_state=19
        )
        df_qa_neg = df_qa[df_qa["labels"].apply(len) != 0].sample(
            sample_size // 2, random_state=19
        )
        return pd.concat([df_qa_pos, df_qa_neg])


In [ ]:
ragtruth_qa = read_ragtruth_qa_from_github(sample_size=20)

##

In [ ]:
ragtruth_qa["n_input_words"] = (
    ragtruth_qa["source_info"].astype(str).str.split().str.len()
    + ragtruth_qa["response"].astype(str).str.split().str.len()
)  # a rough estimate of word count

ragtruth_qa["is_faithful"] = (ragtruth_qa["labels"].apply(len) == 0).astype(int)
display(ragtruth_qa["is_faithful"].value_counts())

## 2. Helper Code for Guardrails and Evaluation Code


In [ ]:
def get_faithfulness_response(
    context: str, response: str, sleep_seconds: float = 0.3
) -> Tuple[float, float]:
    """Invoke the guardrail for Faithfulness, returning the faithfulness score and latency time

    Args:
        context (str): Guardrails context field
        response (str): Guardrails response field

    Returns:
        Tuple[float, float]: Faithfulness score and latency in seconds.
    """
    time.sleep(sleep_seconds)
    data = {
        "context": context,
        "response": response,
    }
    try:
        response = requests.post(
            FAITHFULNESS_GUARDRAILS_URL,
            headers=GUARDRAILS_HEADERS,
            json={"data": data},
            timeout=30,
        )
        response.raise_for_status()
    except Exception as e:
        print(e)
        return None, response.elapsed.total_seconds()

    return response.json()["fdl_faithful_score"], response.elapsed.total_seconds()

In [ ]:
faithful_display_name_map = {0: "hallucination", 1: "faithful"}
pos_label = "faithful"

In [ ]:
def plot_classification_results(df: pd.DataFrame, ds_name: str):
    global faithful_display_name_map, pos_label

    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    pr = PrecisionRecallDisplay.from_predictions(
        df["is_faithful"].map(faithful_display_name_map),
        y_pred=df["fdl_faithful_score"],
        ax=axes[2],
        pos_label=pos_label,
    )
    pr.ax_.set_title(f"Precision-Recall Curve: {ds_name}")

    roc = RocCurveDisplay.from_predictions(
        df["is_faithful"].map(faithful_display_name_map),
        y_pred=df["fdl_faithful_score"],
        ax=axes[1],
        plot_chance_level=True,
        pos_label=pos_label,
    )
    roc.ax_.set_title(f"ROC Curve: {ds_name}")

    axes[0].hist(
        df[df["is_faithful"] == 1]["fdl_faithful_score"],
        color="cornflowerblue",
        label="Faithful",
        bins=np.arange(0, 1.05, 0.1),
    )
    axes[0].hist(
        df[df["is_faithful"] == 0]["fdl_faithful_score"],
        color="orange",
        label="Hallucination",
        histtype="step",
        linewidth=1.5,
        bins=np.arange(0, 1.05, 0.1),
    )
    axes[0].set_xlabel("Score")
    axes[0].set_ylabel("Frequency", axes=axes[0])
    axes[0].set_title(f"Faithfulness Scores for {ds_name}")
    axes[0].legend()
    return fig

In [ ]:
def plot_latency(df: pd.DataFrame, ds_name: str):
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    axes[0].hist(df["fdl_faithful_latency"])
    axes[0].set_title(
        f"Latency for {ds_name}: p95={int(1000 * df['fdl_faithful_latency'].quantile(0.95).round(3))}ms"
    )
    axes[0].set_xlabel("Seconds")
    axes[0].set_ylabel("Frequency")

    axes[1].set_title(
        f"Median: {int((df['n_input_words'] / df['fdl_faithful_latency']).median())} Words per Second"
    )
    axes[1].scatter(x=df["n_input_words"], y=df["fdl_faithful_latency"])
    axes[1].set_xlabel("# of Input Words")
    axes[1].set_ylabel("Seconds")

    return fig

## 3. Run Dataset Through Guardrails

In [ ]:
ragtruth_qa[["fdl_faithful_score", "fdl_faithful_latency"]] = ragtruth_qa.apply(
    lambda row: get_faithfulness_response(str(row["source_info"]), row["response"]),
    axis=1,
    result_type="expand",
)

In [ ]:
plt.show(plot_classification_results(ragtruth_qa, "RAGTruth Q&A"))

In [ ]:
print(
    classification_report(
        ragtruth_qa["is_faithful"].map(faithful_display_name_map),
        (ragtruth_qa["fdl_faithful_score"] > 0.6)
        .astype(int)
        .map(faithful_display_name_map),
    )
)

In [ ]:
rag_latency_fig = plot_latency(ragtruth_qa, "RAGTruth Q&A")
plt.show()
